In [1]:
import datetime as dt
import pandas as pd
import psycopg2
import pytz
import re
import sys
import time
import warnings
from bs4 import BeautifulSoup
from io import StringIO
from psycopg2.extras import execute_values
from psycopg2.extensions import AsIs
from typing import List
from urllib.request import urlopen
import sys
sys.path.insert(0, '../..')
from db_utils import generate_unique_game_id, execute_sql_script
from scrape_all_games_data import get_daily_games_at_date, clean_df, insert_daily_games_at_date
from typing import Callable
from db_utils import load_config

c:\Users\brtnl\OneDrive\Desktop\code\sports_betting\src\cfb\data\scrape_all_games_data.py:46: SyntaxWarning: invalid escape sequence '\d'
  pattern = "^[A-Za-z]{3} \d{1,2}/\d{2}$"


In [2]:
insert_daily_games_at_date(dt.date(2024, 11, 16))


        INSERT INTO cfb.all_games_log(unique_id, date)
        VALUES %s
        ON CONFLICT (unique_id, date) 
        DO UPDATE SET
            all_quarters_scrape = EXCLUDED.all_quarters_scrape
    
[('6e9b5ac413a80f934609303d3df0b3e1', Timestamp('2024-11-16 00:00:00')), ('35d2850a71a90854b8aa46170e16ccd7', Timestamp('2024-11-16 00:00:00')), ('b46374af36f9862abeddba65ffc123de', Timestamp('2024-11-16 00:00:00')), ('488de2e4d3e3cf53997a9c0d789779fe', Timestamp('2024-11-16 00:00:00')), ('338450f6ad57c73ed3e696a0db753fcc', Timestamp('2024-11-16 00:00:00')), ('67aa7a407ce0e2c342bb3175f3b8cb83', Timestamp('2024-11-16 00:00:00')), ('0ce0e3615a35ebffa770c19d17c5b87d', Timestamp('2024-11-16 00:00:00')), ('a43752a85dc8857b18ef084be2cbb507', Timestamp('2024-11-16 00:00:00')), ('2f44b77eb8b8f625e3fc3ec502e2c034', Timestamp('2024-11-16 00:00:00')), ('9b13fe068ddde952bb4092dcb7391a61', Timestamp('2024-11-16 00:00:00')), ('c1c03bafe9bc5957306eca2bd90d81ed', Timestamp('2024-11-16 00:00:00')), ('b1

In [3]:
get_daily_games_at_date(dt.date(2023, 1, 2))

,date,home,home_points,visitor,visitor_points,home_rank,visitor_rank,unique_id
0,2023-01-02,Mississippi State,19,Illinois,10,24,None,fb5fcadb1896e5bd2e4af009bb414907
0,2023-01-02,Penn State,35,Utah,21,9,7,3055bc1f011e34b3009e66b8cdd29512
0,2023-01-02,LSU,63,Purdue,7,16,None,82bb3230a5133a7431dc5e31af947601
0,2023-01-02,Tulane,46,USC,45,14,8,226cc9d3e0961d8f048a5eda4baa45e3


In [4]:
# TODO: what happens if we have overtimes (in future https://www.sports-reference.com/cfb/boxscores/2021-10-23-penn-state.html)
get_table_function = get_daily_games_at_date
params = {
    'date': dt.date(2023, 1, 2)
}
query = """
    INSERT INTO cfb.all_games(date, home, home_points, visitor, visitor_points, home_rank, visitor_rank, unique_id)
    VALUES %s
    ON CONFLICT DO NOTHING
"""

In [5]:
# # def insert_data(get_table_function: Callable, query: str, params: dict): 
# config = load_config()
# db_config = config["database"]
# try:
#     conn = psycopg2.connect(**db_config)
# except:
#     print("Failure to connect to database.")

# all_games_table = get_table_function(**params)
# if all_games_table is None:
#     conn.close()
#     pass

# # Split up each row value into tuples
# row_tuples = [tuple(row) for row in all_games_table.values]
# with conn.cursor() as cursor:
#     try:
#         psycopg2.extras.execute_values(cursor, query, row_tuples)
#         print(
#             "Successfully executed '%(name)s' with params %(params)s"
#             % {"name": get_table_function.__name__, "params": params}
#         )
#     except Exception as e:
#         print(e)
        
# conn.commit()
# conn.close()

In [6]:
get_daily_games_at_date(dt.datetime(2023, 1, 1))

In [7]:
url = "https://www.sports-reference.com/cfb/boxscores/index.cgi?month=1&day=2&year=2023"
html = urlopen(url)
soup = BeautifulSoup(html, features="html.parser")

In [8]:
# # soup.find_all("table", {"class": re.compile("teams")})
# tables = soup.find_all("table", {"class": re.compile("teams")})
# html_string = "\n".join(str(table) for table in tables)
# html_io = StringIO(html_string)


    # pattern = "^[A-Za-z]{3} \d{1,2}/\d{2}$"
    # match_string = df.iloc[0][0]
    # if re.fullmatch(pattern, match_string):
    #     match_string.to_strftime()

    # home_row = df.iloc[1]
    # home, home_rank = get_rank_from_row(home_row)
    # visitor_row = df.iloc[2]
    # visitor, visitor_rank = get_rank_from_row(visitor_row)

In [9]:
def is_valid_date(date_string):
    # Regex pattern for "Month Day, Year" with an optional day of the week
    pattern = r'^(?:\w{3,9},\s)?\w{3,9}\s\d{1,2},\s\d{4}$'
    return bool(re.match(pattern, date_string))

def get_rank_from_row(row):
    pattern2 = r"^(.*?)\s\((\d{1,2})\)$"
    university = row[0]
    match = re.fullmatch(pattern2, university)
    if match:
         return match.group(1), int(match.group(2))
    else:
         return university, None

cand_divs = soup.find_all(attrs={"class": re.compile("game_summaries")})
date, div = None, None
date_format = "%A, %B %d, %Y"


for cand_div in cand_divs:
    h2_text = cand_div.find('h2').text
    if is_valid_date(h2_text):
        date = dt.datetime.strptime(h2_text, date_format)
        html_io = StringIO(str(cand_div))
        all_games_table = pd.read_html(html_io)

# have the desired table, no "other games this week"

In [10]:
# html_string = "\n".join(str(table) for table in tables)
# html_io = StringIO(html_string)

# all_games_table = pd.read_html(html_io, header=0)[0]

In [11]:
# [div for div in tables if div.find_parent('div', {"class": re.compile("game_summaries")})]

In [12]:
cand_divs = soup.find_all(attrs={"class": re.compile("game_summaries")})
date_format = "%A, %B %d, %Y"

for cand_div in cand_divs:
    h2_text = cand_div.find('h2').text
    if is_valid_date(h2_text):
        date = dt.datetime.strptime(h2_text, date_format)
        html_io = StringIO(str(cand_div))
        all_games_table = pd.read_html(html_io)
        break


for cand_div in cand_divs:
    h2_text = cand_div.find('h2').text
    if is_valid_date(h2_text):
        date = dt.datetime.strptime(h2_text, date_format)
        html_io = StringIO(str(cand_div))
        all_games_table = pd.read_html(html_io)
        
def get_rank_from_row(row):
    pattern2 = r"^(.*?)\s\((\d{1,2})\)$"
    university = row[0]
    match = re.fullmatch(pattern2, university)
    if match:
         return match.group(1), int(match.group(2))
    else:
         return university, None
    

def clean_df(df: pd.DataFrame, date: dt.datetime): 
    pattern = "^[A-Za-z]{3} \d{1,2}/\d{2}$"
    match_string = df.iloc[0][0]
    if re.fullmatch(pattern, match_string):
        match_string.to_strftime()

    home_row = df.iloc[1]
    home, home_rank = get_rank_from_row(home_row)
    home_points = home_row[1]
    visitor_row = df.iloc[2]
    visitor, visitor_rank = get_rank_from_row(visitor_row)
    visitor_points = visitor_row[1]
    
    id_row = pd.Series({
        'Date': [date],
        'Home': [home],
        'Visitor': [visitor]
    })
    
    return pd.DataFrame({
        'date': [date],
        'home': [home],
        'home_points': [home_points],
        'visitor': [visitor],
        'visitor_points': [visitor_points],
        'home_rank': [home_rank],
        'visitor_rank': [visitor_rank],
        'unique_id': [generate_unique_game_id(id_row)]
    })

<>:31: SyntaxWarning: invalid escape sequence '\d'
<>:31: SyntaxWarning: invalid escape sequence '\d'
C:\Users\brtnl\AppData\Local\Temp\ipykernel_15856\1596917811.py:31: SyntaxWarning: invalid escape sequence '\d'
  pattern = "^[A-Za-z]{3} \d{1,2}/\d{2}$"


In [13]:
game_list = []
for game in all_games_table:
    game_list.append(clean_df(game, date))

In [14]:
pd.concat(game_list)

,date,home,home_points,visitor,visitor_points,home_rank,visitor_rank,unique_id
0,2023-01-02,Mississippi State,19,Illinois,10,24,None,fb5fcadb1896e5bd2e4af009bb414907
0,2023-01-02,Penn State,35,Utah,21,9,7,3055bc1f011e34b3009e66b8cdd29512
0,2023-01-02,LSU,63,Purdue,7,16,None,82bb3230a5133a7431dc5e31af947601
0,2023-01-02,Tulane,46,USC,45,14,8,226cc9d3e0961d8f048a5eda4baa45e3


In [15]:
temp = all_games_table[1]
# temp.drop_index(inplace=True)
clean_df(temp)

TypeError: clean_df() missing 1 required positional argument: 'date'

In [ ]:
all_games_table

[          0   1      2
 0  USC (23)  27  Final
 1  LSU (13)  20    NaN,
                     0        1        2
 0             Mon 9/2  Mon 9/2  Mon 9/2
 1      Boston College       28    Final
 2  Florida State (10)       13      NaN,
            0         1         2
 0   Sat 8/31  Sat 8/31  Sat 8/31
 1  Merrimack         6     Final
 2  Air Force        21       NaN,
                   0         1         2
 0          Sat 8/31  Sat 8/31  Sat 8/31
 1  Western Kentucky         0     Final
 2              None      None      None,
    index                     0         1         2
 0      0              Sat 8/31  Sat 8/31  Sat 8/31
 1      1  East Tennessee State        10     Final
 2      2     Appalachian State        38       NaN,
               0         1         2
 0      Sat 8/31  Sat 8/31  Sat 8/31
 1    New Mexico        39     Final
 2  Arizona (21)        61       NaN,
                0         1         2
 0       Sat 8/31  Sat 8/31  Sat 8/31
 1        Wyoming         

In [ ]:
def get_rank_from_row(row):
    pattern2 = r"^(.*?)\s\((\d{1,2})\)$"
    university = row[0]
    match = re.fullmatch(pattern2, university)
    if match:
         return match.group(1), int(match.group(2))
    else:
         return university, None

In [ ]:
get_rank_from_row(all_games_table[5].iloc[2])

('Arizona', '21')

In [ ]:
all_games_table[5]

,0,1,2
0,Sat 8/31,Sat 8/31,Sat 8/31
1,New Mexico,39,Final
2,Arizona (21),61,NaN


In [ ]:
temp.iloc[0][0]

'Western Kentucky'

In [ ]:
regex2 = r".*\(\b([1-9]|1[0-9]|2[0-5])\b\)"
re.fullmatch(regex2, "Western Kentucky (5)")

<re.Match object; span=(0, 20), match='Western Kentucky (5)'>

In [ ]:
temp = all_games_table[3][1:]
temp.loc[:, "rank"] = None
# temp

/var/folders/z_/5kxjclhd5090czxnlnxslydr0000gn/T/ipykernel_88211/1111760236.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp.loc[:, "rank"] = None


regex = "^[A-Za-z]{3} \d{1,2}/\d{2}$"
string = all_games_table[3].iloc[0][0]

s = re.fullmatch(regex, string)

In [ ]:
s

<re.Match object; span=(0, 8), match='Sat 8/31'>

In [ ]:
month = month.lower()

    # URL to scrape, notice f string:
    url = (
        f"https://www.basketball-reference.com/leagues/NBA_%(year)s_games-%(month)s.html"
        % {"year": year, "month": month}
    )
    try:
        html = urlopen(url)
    except Exception as e:
        print(e)
        print(
            "Failure: %(month)s, %(year)s all_games URL not pulled"
            % {"month": month.capitalize(), "year": year}
        )
        return None

    # Kicks you out if you request over 20 times over a minute
    time.sleep(3.5)

    soup = BeautifulSoup(html, features="html.parser")

    # Convert the tables into a string and wrap it with StringIO
    tables = soup.find_all("table", {"id": re.compile("schedule")})
    html_string = "\n".join(str(table) for table in tables)
    html_io = StringIO(html_string)

    all_games_table = pd.read_html(html_io, header=0)[0]
    all_games_table = clean_all_games_table(all_games_table)
    if all_games_table is not None:
        all_games_table["Unique_ID"] = all_games_table.apply(
            generate_unique_game_id, axis=1
        )
    return all_games_table